In [1]:
import tensorflow as tf
import experiment_utils as eu
import sys
sys.path.append('../')
from data_processing.stl10 import get_stl10
import importlib
import config as conf

In [2]:
importlib.reload(eu)
ds, _ = get_stl10(batch_size=5, split='train')
ds_small = ds.take(5)
config = conf.get_byol()
experiment = eu.Experiment(config)
experiment.train(ds_small, 'saved_model', epochs=1)


varliables target: 
 [<tf.Variable 'res_net18_1/conv2d_20/kernel:0' shape=(7, 7, 3, 64) dtype=float32, numpy=
array([[[[ 7.13101402e-03, -1.08212642e-02,  1.75425187e-02, ...,
          -3.01801339e-02, -3.75212952e-02,  1.48176067e-02],
         [-4.04120348e-02, -3.41520049e-02, -1.55382063e-02, ...,
           2.18393765e-02,  2.26096027e-02,  2.23434456e-02],
         [-1.96427219e-02,  2.04371326e-02, -1.81615800e-02, ...,
           9.32507962e-03, -2.59556733e-02,  3.86325084e-02]],

        [[ 3.78599763e-02, -3.02022733e-02,  1.68009736e-02, ...,
          -3.21607664e-03, -2.59746723e-02, -2.19756402e-02],
         [ 7.17030466e-03, -1.77399572e-02,  3.10802460e-03, ...,
          -1.27667766e-02,  4.18871231e-02, -1.65146962e-02],
         [-7.09977001e-04, -2.98420079e-02,  1.42942630e-02, ...,
          -1.77808851e-03, -1.60246324e-02, -3.42210755e-03]],

        [[-1.23124570e-04, -1.88492872e-02, -3.72549556e-02, ...,
          -2.63853688e-02,  4.02253829e-02,  2.29878

  0%|          | 0/5 [00:00<?, ?it/s]

[<tf.Variable 'res_net18/conv2d/kernel:0' shape=(7, 7, 3, 64) dtype=float32, numpy=
array([[[[ 7.13101402e-03, -1.08212642e-02,  1.75425187e-02, ...,
          -3.01801339e-02, -3.75212952e-02,  1.48176067e-02],
         [-4.04120386e-02, -3.41520049e-02, -1.55382063e-02, ...,
           2.18393765e-02,  2.26096027e-02,  2.23434456e-02],
         [-1.96427219e-02,  2.04371326e-02, -1.81615800e-02, ...,
           9.32507962e-03, -2.59556733e-02,  3.86325084e-02]],

        [[ 3.78599800e-02, -3.02022733e-02,  1.68009736e-02, ...,
          -3.21607664e-03, -2.59746723e-02, -2.19756402e-02],
         [ 7.17030466e-03, -1.77399572e-02,  3.10802460e-03, ...,
          -1.27667766e-02,  4.18871231e-02, -1.65146962e-02],
         [-7.09977001e-04, -2.98420079e-02,  1.42942630e-02, ...,
          -1.77808851e-03, -1.60246324e-02, -3.42210755e-03]],

        [[-1.23124570e-04, -1.88492872e-02, -3.72549556e-02, ...,
          -2.63853688e-02,  4.02253829e-02,  2.29878537e-02],
         [ 3.640

100%|██████████| 5/5 [00:18<00:00,  3.76s/it]


Epoch 000: Loss: 0.824


KeyboardInterrupt: 

In [15]:
import numpy as np
test = np.array(experiment.test[0])
print(test.shape)
test

(5, 128)


array([[ 3.31537202e-02, -5.36188558e-02, -1.87725574e-02,
         5.00399433e-02, -6.07553916e-03,  1.54159106e-02,
         4.35693860e-02,  5.63887060e-02, -2.96695232e-02,
         9.03071836e-03,  5.76402545e-02,  5.48327342e-02,
         2.57254932e-02, -3.85051034e-03,  2.17803158e-02,
        -3.79391015e-04,  4.20477986e-02,  7.78120477e-03,
        -1.62516944e-02, -7.28299469e-02, -6.71901740e-04,
        -8.38991255e-05,  3.69728096e-02,  7.65915634e-03,
        -1.66247226e-03, -4.38600741e-02,  1.05069047e-02,
         1.80620179e-02,  3.87496836e-02, -2.79563610e-02,
        -4.26811948e-02, -6.25194237e-03, -4.92621548e-02,
        -4.89186607e-02,  2.87935827e-02, -1.71239190e-02,
         4.41903137e-02,  3.95971090e-02, -4.88426350e-02,
         3.55689460e-03,  2.83920784e-02, -7.92761706e-03,
         1.09058619e-02, -3.30093987e-02, -3.07188965e-02,
         4.03139368e-03,  3.47156487e-02,  4.15029377e-03,
         2.20839325e-02, -1.06048528e-02, -2.68328236e-0

In [20]:
import tensorflow as tf
tfm = tf.math
test = tfm.l2_normalize(test, axis=-1)
tfm.reduce_sum(test, axis=-1)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.6404567 , -0.45835656, -0.47579864, -0.5814763 , -0.68898994],
      dtype=float32)>

In [5]:
import tensorflow_datasets as tfds
ds, ds_info = tfds.load(
    'stl10', split='unlabelled', with_info=True, as_supervised=True)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label


def resize_to_resnet_input(image, label):
    """Resizes images to resnet compatible size (224x224)."""
    return tf.image.resize(image, [224, 224]), label

ds = ds.map(normalize_img,  num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.map(resize_to_resnet_input,  num_parallel_calls=tf.data.AUTOTUNE)
# TODO: for now that's only for pretraining.
# ds = ds.map(lambda img, _: img,  num_parallel_calls=tf.data.AUTOTUNE)

# ds = ds.cache()
# For true randomness, set the shuffle buffer to the full dataset size.
ds = ds.shuffle(1000)
ds = ds.batch(128)

ds = ds.prefetch(tf.data.AUTOTUNE)

for a, b in ds.take(10):
    print(a)

tf.Tensor(
[[[[0.16078432 0.2784314  0.5058824 ]
   [0.16190477 0.2784314  0.50756305]
   [0.16526611 0.2784314  0.5126051 ]
   ...
   [0.22072832 0.34733894 0.5501401 ]
   [0.22577035 0.34565824 0.55518216]
   [0.22745098 0.34509805 0.5568628 ]]

  [[0.16134454 0.27787116 0.5086835 ]
   [0.16230492 0.27795118 0.5096439 ]
   [0.16518608 0.2781913  0.512525  ]
   ...
   [0.2219288  0.34757903 0.5519808 ]
   [0.22649063 0.34613845 0.5560625 ]
   [0.2280112  0.34565827 0.557423  ]]

  [[0.16302522 0.2761905  0.51708686]
   [0.1635054  0.27651063 0.51588637]
   [0.16494599 0.277471   0.51228493]
   ...
   [0.22553024 0.34829932 0.55750304]
   [0.2286515  0.34757903 0.55870354]
   [0.22969188 0.34733894 0.55910367]]

  ...

  [[0.3154061  0.3507001  0.292997  ]
   [0.29147652 0.31476575 0.26538596]
   [0.21968776 0.20696265 0.18255284]
   ...
   [0.16182502 0.1238098  0.12372968]
   [0.23457438 0.18599492 0.16454615]
   [0.2588233  0.20672254 0.17815116]]

  [[0.30532205 0.31708652 0.252660